In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [65]:
df = pd.read_csv("Artificats/Time_Estimation_Dataset/train.csv")

In [66]:
df.head()

,ID,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken
0,0x4607,37.0,4.9,22.765049,75.912471,2022-03-19,11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24
1,0xb379,34.0,4.5,13.043041,77.813237,2022-03-25,19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33
2,0x5d6d,23.0,4.4,12.924264,77.688400,2022-03-19,08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26
3,0x7a6a,38.0,4.7,11.053669,77.026494,2022-04-05,18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21
4,0x70a2,32.0,4.6,13.012793,80.289982,2022-03-26,13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30


In [67]:
df.drop(['ID'], axis=1, inplace=True)

In [68]:
df['Order_Date']=pd.to_datetime(df['Order_Date'])

# Creating three column for day,month and year
df['Order_day']=df['Order_Date'].dt.day
df['Order_month']=df['Order_Date'].dt.month
df['Order_year']=df['Order_Date'].dt.year

In [69]:
df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])

# Creating two new column for hour and minute
df['Hour_order']=df['Time_Orderd'].dt.hour
df['Min_order']=df['Time_Orderd'].dt.minute

C:\Users\rahul\AppData\Local\Temp\ipykernel_15980\1731031211.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])


In [70]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken,Order_day,Order_month,Order_year,Hour_order,Min_order
0,37.0,4.9,22.765049,75.912471,2022-03-19,2025-01-17 11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24,19,3,2022,11.0,30.0
1,34.0,4.5,13.043041,77.813237,2022-03-25,2025-01-17 19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33,25,3,2022,19.0,45.0
2,23.0,4.4,12.924264,77.688400,2022-03-19,2025-01-17 08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26,19,3,2022,8.0,30.0
3,38.0,4.7,11.053669,77.026494,2022-04-05,2025-01-17 18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21,5,4,2022,18.0,0.0
4,32.0,4.6,13.012793,80.289982,2022-03-26,2025-01-17 13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30,26,3,2022,13.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,2022-03-24,2025-01-17 11:35:00,Windy,High,1,motorcycle,0.0,Metropolitian,32,24,3,2022,11.0,35.0
45589,21.0,4.6,0.070000,0.070000,2022-02-16,2025-01-17 19:55:00,Windy,Jam,0,motorcycle,1.0,Metropolitian,36,16,2,2022,19.0,55.0
45590,30.0,4.9,13.052394,80.272439,2022-03-11,2025-01-17 23:50:00,Cloudy,Low,1,scooter,0.0,Metropolitian,16,11,3,2022,23.0,50.0
45591,20.0,4.7,11.041753,77.026241,2022-03-07,2025-01-17 13:35:00,Cloudy,High,0,motorcycle,1.0,Metropolitian,26,7,3,2022,13.0,35.0


In [71]:
df.drop(["Time_Orderd", "Order_Date"],axis = 1, inplace= True)

In [72]:
df.shape

(45593, 16)

In [73]:
column = df.pop("Time_taken")
df.insert(15, column.name, column)

In [75]:
def create_delivery_features(df):

    df_features = df
    
    df_features['avg_delivery_time_area'] = df_features.groupby('City')['Time_taken'].transform('mean')

    df_features['traffic_weather_impact'] = df_features.groupby(
        ['Road_traffic_density', 'Weatherconditions']
    )['Time_taken'].transform('mean')

    df_features['vehicle_capacity_utilization'] = (
        df_features['multiple_deliveries'] / 
        df_features.groupby('Type_of_vehicle')['multiple_deliveries'].transform('max')
    ).fillna(0)
    
    return df_features
    
df = create_delivery_features(df)

In [79]:
col = df.pop("Time_taken")
df.insert(18, col.name, col)

In [82]:
df1 = df.copy()

In [80]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Order_day,Order_month,Order_year,Hour_order,Min_order,avg_delivery_time_area,traffic_weather_impact,vehicle_capacity_utilization,Time_taken
0,37.0,4.9,22.765049,75.912471,Sunny,High,2,motorcycle,0.0,Urban,19,3,2022,11.0,30.0,22.984017,23.448980,0.000000,24
1,34.0,4.5,13.043041,77.813237,Stormy,Jam,2,scooter,1.0,Metropolitian,25,3,2022,19.0,45.0,27.315226,29.850194,0.333333,33
2,23.0,4.4,12.924264,77.688400,Sandstorms,Low,0,motorcycle,1.0,Urban,19,3,2022,8.0,30.0,22.984017,20.297049,0.333333,26
3,38.0,4.7,11.053669,77.026494,Sunny,Medium,0,motorcycle,1.0,Metropolitian,5,4,2022,18.0,0.0,27.315226,20.195518,0.333333,21
4,32.0,4.6,13.012793,80.289982,Cloudy,High,1,scooter,1.0,Metropolitian,26,3,2022,13.0,30.0,27.315226,28.940860,0.333333,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,Windy,High,1,motorcycle,0.0,Metropolitian,24,3,2022,11.0,35.0,27.315226,26.972789,0.000000,32
45589,21.0,4.6,0.070000,0.070000,Windy,Jam,0,motorcycle,1.0,Metropolitian,16,2,2022,19.0,55.0,27.315226,30.219056,0.333333,36
45590,30.0,4.9,13.052394,80.272439,Cloudy,Low,1,scooter,0.0,Metropolitian,11,3,2022,23.0,50.0,27.315226,22.208445,0.000000,16
45591,20.0,4.7,11.041753,77.026241,Cloudy,High,0,motorcycle,1.0,Metropolitian,7,3,2022,13.0,35.0,27.315226,28.940860,0.333333,26


In [84]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [60]:
# def create_delivery_features(df):
#     """
#     Create new features from existing delivery data
    
#     Parameters:
#     df (pandas.DataFrame): Input dataframe with delivery data
    
#     Returns:
#     pandas.DataFrame: DataFrame with new features added
#     """
#     # Create a copy to avoid modifying the original dataframe
#     df_features = df.copy()
    
#     # 1. Average delivery time by area
#     # Calculate area-wise average delivery time
#     area_avg_time = df_features.groupby('City')['Time_taken'].transform('mean')
#     df_features['area_avg_delivery_time'] = area_avg_time
    
#     # Calculate delivery time deviation from area average
#     df_features['delivery_time_vs_area_avg'] = df_features['Time_taken'] - area_avg_time
    
#     # 2. Impact of traffic and weather on delivery time
#     # Create traffic impact score (Higher score means more impact)
#     traffic_mapping = {
#         'Low': 1,
#         'Medium': 2,
#         'High': 3,
#         'Jam': 4
#     }
#     df_features['traffic_impact_score'] = df_features['Road_traffic_density'].map(traffic_mapping)
    
#     # Create weather impact score (Based on severity of weather condition)
#     weather_mapping = {
#         'Sunny': 1,
#         'Cloudy': 2,
#         'Stormy': 4,
#         'Sandstorms': 5
#     }
#     df_features['weather_impact_score'] = df_features['Weather_conditions'].map(weather_mapping)
    
#     # Combined weather and traffic impact
#     df_features['combined_condition_impact'] = df_features['traffic_impact_score'] * df_features['weather_impact_score']
    
#     # Normalize the impact scores
#     df_features['normalized_condition_impact'] = df_features['combined_condition_impact'] / df_features['combined_condition_impact'].max()
    
#     # 3. Vehicle capacity utilization
#     # Calculate deliveries per vehicle
#     vehicle_deliveries = df_features.groupby('Type_of_vehicle')['multiple_deliveries'].transform('mean')
#     df_features['vehicle_avg_deliveries'] = vehicle_deliveries
    
#     # Calculate vehicle utilization score
#     df_features['vehicle_utilization'] = (df_features['multiple_deliveries'] + 1) / (df_features['vehicle_avg_deliveries'].max() + 1)
    
#     # Calculate delivery efficiency (time per delivery)
#     df_features['delivery_efficiency'] = df_features['Time_taken'] / (df_features['multiple_deliveries'] + 1)
    
#     # Vehicle condition impact on utilization
#     df_features['vehicle_condition_score'] = df_features['Vehicle_condition'] / df_features['Vehicle_condition'].max()
    
#     # Calculate overall delivery performance score
#     df_features['delivery_performance_score'] = (
#         (1 - df_features['normalized_condition_impact']) * 0.3 +  # Lower impact is better
#         df_features['vehicle_utilization'] * 0.3 +               # Higher utilization is better
#         df_features['vehicle_condition_score'] * 0.2 +           # Better condition is better
#         (1 - df_features['delivery_time_vs_area_avg'].clip(lower=0) / 
#          df_features['delivery_time_vs_area_avg'].clip(lower=0).max()) * 0.2  # Lower delivery time vs avg is better
#     )
    
#     # Round all numeric columns to 3 decimal places
#     numeric_columns = df_features.select_dtypes(include=[np.number]).columns
#     df_features[numeric_columns] = df_features[numeric_columns].round(3)
    
#     return df_features

# dfe = create_delivery_features(df)

KeyError: 'Weather_conditions'